In [5]:
!pip install pyvis

In [10]:
import pandas as pd
import math
from pyvis.network import Network

# Load your CSV
df = pd.read_csv("INDIVIDUALS_EVOLVING_EMOTE_6/all_individuals_evaluation_log.csv")

net = Network(height="900px", width="100%", directed=True, notebook=False)

# Disable physics completely (otherwise nodes will float around)
net.toggle_physics(False)

# Limit to top 50 by fitness per generation
top_nodes = []
for gen, group in df.groupby("generation"):
    top = group.nlargest(50, "fitness")
    top_nodes.append(top)
top_df = pd.concat(top_nodes)

# --- Add INITIAL node ---
initial_img = "INITIAL.png"
initial_label = "INITIAL"
initial_scores = {
    "angry": 0.47, "disgust": 0.0, "fear": 0.99, "happy": 0.04,
    "sad": 1.22, "surprise": 0.30, "neutral": 96.95
}
initial_html = f"""
<div style="text-align:center; font-size:12px;">
    <img src="{initial_img}" width="120"><br>
    <b>{initial_label}</b><br>
    Angry: {initial_scores['angry']:.2f} | Disgust: {initial_scores['disgust']:.2f}<br>
    Fear: {initial_scores['fear']:.2f} | Happy: {initial_scores['happy']:.2f}<br>
    Sad: {initial_scores['sad']:.2f} | Surprise: {initial_scores['surprise']:.2f}<br>
    Neutral: {initial_scores['neutral']:.2f}
</div>
"""
net.add_node("INITIAL", title=initial_html, label=initial_label,
             shape="image", image=initial_img, x=0, y=0, fixed=True)

# --- Add face nodes with manual positioning ---
gen_x_gap = 600   # horizontal spacing between generations
gen_y_gap = 75   # vertical spacing between nodes

for gen, group in top_df.groupby("generation"):
    nodes = group.to_dict("records")
    n = len(nodes)
    grid_size = math.ceil(math.sqrt(n))  # rows/cols in the square grid
    
    for i, row in enumerate(nodes):
        indiv_id = row["individual_id"]
        label = f"output_gen{gen}_{indiv_id[:8]}"
        img = row["face_pic"]

        node_html = f"""
        <div style="text-align:center; font-size:12px;">
            <img src="{img}" width="120"><br>
            <b>{label}</b><br>
            Angry: {row['angry']:.2f} | Disgust: {row['disgust']:.2f}<br>
            Fear: {row['fear']:.2f} | Happy: {row['happy']:.2f}<br>
            Sad: {row['sad']:.2f} | Surprise: {row['surprise']:.2f}<br>
            Neutral: {row['neutral']:.2f}
        </div>
        """

        # # Compute grid position
        # row_i = i // grid_size
        # col_i = i % grid_size
        # x = gen * gen_x_gap
        # y = (row_i - grid_size/2) * gen_y_gap + col_i * 5  # small offset to break perfect lines
        # Compute grid position in a square layout
        row_i = i // grid_size  # row index
        col_i = i % grid_size   # column index

        # Center the grid around the generation's x-position
        x = gen * gen_x_gap + (col_i - grid_size / 2) * gen_x_gap / 2
        y = (row_i - grid_size / 2) * gen_y_gap
        
        net.add_node(
            indiv_id,
            title=node_html,
            label=label,
            shape="image",
            image=img,
            x=x,
            y=y,
            fixed=True
        )

# --- Add edges ---
valid_nodes = set(top_df["individual_id"]) | {"INITIAL"}
for _, row in top_df.iterrows():
    for parent in [row["parent1_individual_id"], row["parent2_individual_id"]]:
        if parent in valid_nodes:
            net.add_edge(parent, row["individual_id"])

# Some prettier options (curved edges, smooth transitions, better interaction)
net.set_options("""
{
  "edges": {
    "smooth": {
      "type": "curvedCW",
      "roundness": 0.2
    }
  },
  "nodes": {
    "borderWidth": 2,
    "shadow": true
  },
  "interaction": {
    "dragNodes": true,
    "zoomView": true
  },
  "physics": {
    "enabled": false
  }
}
""")

# Export to HTML
net.write_html("faces_lineage.html", notebook=False)


In [ ]:
# import pandas as pd
# from pyvis.network import Network

# # Load your CSV
# df = pd.read_csv("INDIVIDUALS_EVOLVING_EMOTE_5/all_individuals_evaluation_log.csv")

# # Initialize network
# net = Network(height="900px", width="100%", directed=True, notebook=False)
# net.toggle_physics(True)  # start with physics on (toggleable in UI)

# # Limit to top 50 by fitness per generation
# top_nodes = []
# for gen, group in df.groupby("generation"):
#     top = group.nlargest(50, "fitness")
#     top_nodes.append(top)
# top_df = pd.concat(top_nodes)

# # --- Add INITIAL node ---
# # You can customize this however you want
# initial_img = "INITIAL.png"  # put your actual file path here
# initial_scores = {
#     "angry": 0.47879128776736785,
#     "disgust": 1.5892657027317155e-07,
#     "fear": 0.9973007848740092,
#     "happy": 0.04908755364977143,
#     "sad": 1.2205196340691826,
#     "surprise": 0.30427833038737356,
#     "neutral": 96.95002418458606
# }

# initial_label = "INITIAL"

# initial_html = f"""
# <div style="text-align:center; font-size:12px;">
#     <img src="{initial_img}" width="120"><br>
#     <b>{initial_label}</b><br>
#     Angry: {initial_scores['angry']:.2f} | Disgust: {initial_scores['disgust']:.2f}<br>
#     Fear: {initial_scores['fear']:.2f} | Happy: {initial_scores['happy']:.2f}<br>
#     Sad: {initial_scores['sad']:.2f} | Surprise: {initial_scores['surprise']:.2f}<br>
#     Neutral: {initial_scores['neutral']:.2f}
# </div>
# """

# net.add_node(
#     "INITIAL",
#     title=initial_html,
#     label=initial_label,
#     shape="image",
#     image=initial_img
# )

# # --- Add face nodes ---
# for _, row in top_df.iterrows():
#     indiv_id = row["individual_id"]
#     label = f"output_gen{row['generation']}_{indiv_id[:8]}"
#     img = row["face_pic"]
    
#     node_html = f"""
#     <div style="text-align:center; font-size:12px;">
#         <img src="{img}" width="120"><br>
#         <b>{label}</b><br>
#         Angry: {row['angry']:.2f} | Disgust: {row['disgust']:.2f}<br>
#         Fear: {row['fear']:.2f} | Happy: {row['happy']:.2f}<br>
#         Sad: {row['sad']:.2f} | Surprise: {row['surprise']:.2f}<br>
#         Neutral: {row['neutral']:.2f}
#     </div>
#     """
    
#     net.add_node(
#         indiv_id,
#         title=node_html,   # tooltip
#         label=label,       # visible under node
#         shape="image",
#         image=img
#     )

# # --- Add edges ---
# for _, row in top_df.iterrows():
#     for parent in [row["parent1_individual_id"], row["parent2_individual_id"]]:
#         if parent in set(top_df["individual_id"]) or parent == "INITIAL":
#             net.add_edge(parent, row["individual_id"])

# # Export to HTML
# # net.show("faces_lineage.html")
# net.write_html("faces_lineage.html", notebook=False)
